We’ll use one table for all symbols — simpler for queries & backtesting.



## New Workflow:
1. Write multiple ASX tickers into a single CSV
2. Save CSV to sqlite and work around the pk constraint to allow loading same file to update any new items and overwrite any existing

Why this format is ideal

✔ One file
✔ Multiple stocks
✔ Easy import into SQLite / Pandas / Excel
✔ Perfect for backtesting & journaling

In [73]:
'''
Why this works

df["date"].dt.strftime("%Y-%m-%d") → converts Timestamp to plain string "2024-01-02"

SQLite is happy with strings for DATE columns

INSERT OR IGNORE skips duplicates (primary key enforcement)

'''

import yfinance as yf
import pandas as pd

# done 2015-1-1 to 2026-1-1

# TICKERS = ["BHP.AX", "CBA.AX", "CSL.AX", "WES.AX", "VAS.AX", "A200.AX","VAP.AX","VHY.AX","NDQ.AX","SPY.AX"]

# Update a single stock
TICKERS = ["BOE.AX"]

all_data = []

for ticker in TICKERS:
    df = yf.download(
        ticker,
        start="2015-01-01",
        auto_adjust=True,
        progress=False
    )

    if df.empty:
        continue

    df.reset_index(inplace=True)
    df["symbol"] = ticker

    df = df[["symbol", "Date", "Open", "High", "Low", "Close", "Volume"]]
    df.columns = ["symbol", "date", "open", "high", "low", "close", "volume"]

    all_data.append(df)

# Combine all tickers into one DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Write to single CSV
final_df.to_csv("asx_all_prices.csv", index=False)

# remove duplicates:
df.drop_duplicates(subset=["symbol", "date"], inplace=True)

print("Saved asx_all_prices.csv")


Saved asx_all_prices.csv


# Optional upgrades (recommended)
## Sort nicely

In [68]:
final_df.to_csv("asx_all_prices.csv", mode="a", header=False, index=False)

In [74]:
# Add daily return:

final_df["daily_return"] = final_df.groupby("symbol")["close"].pct_change()*100

# chekc dataframe:

final_df

,symbol,date,open,high,low,close,volume,daily_return
0,BOE.AX,2015-01-02,0.080816,0.080816,0.080816,0.080816,56273,NaN
1,BOE.AX,2015-01-05,0.080816,0.080816,0.080816,0.080816,0,0.000000
2,BOE.AX,2015-01-06,0.080816,0.080816,0.080816,0.080816,0,0.000000
3,BOE.AX,2015-01-07,0.080816,0.080816,0.080816,0.080816,4491,0.000000
4,BOE.AX,2015-01-08,0.080816,0.080816,0.080816,0.080816,59965,0.000000
...,...,...,...,...,...,...,...,...
2800,BOE.AX,2026-01-23,1.830000,1.875000,1.805000,1.870000,8606702,3.030300
2801,BOE.AX,2026-01-27,1.850000,1.850000,1.790000,1.800000,8677973,-3.743318
2802,BOE.AX,2026-01-28,1.910000,2.010000,1.900000,1.980000,19931746,10.000004
2803,BOE.AX,2026-01-29,2.080000,2.150000,1.942500,2.050000,19988441,3.535350


In [47]:
# read out the csv
df = pd.read_csv("asx_all_prices.csv")
df["daily_return"] = df.groupby("symbol")["close"].pct_change()*100# Write to single CSV

print(df.tail(20))

      symbol        date       open       high        low      close   volume  \
2164  WES.AX  2026-01-02  81.050003  81.845001  80.900002  81.720001   493186   
2165  WES.AX  2026-01-05  81.120003  81.635002  80.839996  80.839996   721797   
2166  WES.AX  2026-01-06  80.500000  80.769997  79.779999  79.779999  1235150   
2167  WES.AX  2026-01-07  80.099998  80.699997  79.489998  79.980003   939192   
2168  WES.AX  2026-01-08  80.419998  81.269997  79.940002  80.959999  1042460   
2169  WES.AX  2026-01-09  81.330002  81.709999  80.730003  81.059998  1040311   
2170  WES.AX  2026-01-12  81.239998  82.400002  81.230003  82.230003  1080104   
2171  WES.AX  2026-01-13  81.430000  82.940002  81.349998  82.000000  1036329   
2172  WES.AX  2026-01-14  81.790001  82.790001  81.680000  82.290001  1002324   
2173  WES.AX  2026-01-15  82.940002  83.150002  82.180000  82.989998  1029256   
2174  WES.AX  2026-01-16  83.000000  83.235001  82.209999  83.209999  1497105   
2175  WES.AX  2026-01-19  83

# CSV to sqlite

# can't use df.to.sql() to insert as it will enforce pk, which in turn prevent update overriding existing data when add new data

Perfect — this is a very common + sensible pipeline:

Yahoo → one CSV → SQLite

Below is a clean, safe implementation that:

Reads the single CSV

Inserts into your existing prices table

Avoids duplicates

Matches the schema exactly

In [75]:
import pandas as pd
import sqlite3

# Read CSV
df = pd.read_csv("asx_all_prices.csv", parse_dates=["date"])

# Convert datetime to string in YYYY-MM-DD format
df["date"] = df["date"].dt.strftime("%Y-%m-%d")

# Connect to SQLite
conn = sqlite3.connect("asx_prices.db")

# create table if not exits
conn.execute("""
        CREATE TABLE IF NOT EXISTS prices (
            symbol TEXT NOT NULL,
            date   DATE NOT NULL,
            open   REAL,
            high   REAL,
            low    REAL,
            close  REAL,
            volume INTEGER,
            PRIMARY KEY (symbol, date)
        )
    """)


cursor = conn.cursor()

# Insert rows safely, ignoring duplicates
cursor.executemany("""
    INSERT OR IGNORE INTO prices
    (symbol, date, open, high, low, close, volume)
    VALUES (?, ?, ?, ?, ?, ?, ?)
""", df.itertuples(index=False, name=None))

conn.commit()
conn.close()

print("CSV loaded into SQLite successfully!")


CSV loaded into SQLite successfully!


In [76]:
# Quick sanity check

conn = sqlite3.connect("asx_prices.db")

pd.read_sql("""
    SELECT symbol, COUNT(*) rows
    FROM prices
    GROUP BY symbol
""", conn)

,symbol,rows
0,A200.AX,1959
1,BHP.AX,2805
2,BOE.AX,2805
3,CBA.AX,2805
4,CSL.AX,2805
5,NDQ.AX,2705
6,SPY.AX,2801
7,VAP.AX,2803
8,VAS.AX,2803
9,VHY.AX,2803


Here’s the clean, correct SQLite query for a date range + specific ticker, plus a Pandas-friendly version since you’re using Python.

In [77]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("asx_prices.db")

# filter parameters:

ticker = "BOE.AX"
start  = "2019-01-01"
end    = "2019-01-12"

# query with parameter

df = pd.read_sql(
    """
    SELECT date, open, high, low, close, volume
    FROM prices
    WHERE symbol = ?
      AND date BETWEEN ? AND ?
    ORDER BY date
    """,
    conn,
    params=(ticker, start, end),
    parse_dates=["date"]
)

conn.close()

print(df.head())


        date   open   high    low  close  volume
0 2019-01-02  0.456  0.464  0.456  0.464   45006
1 2019-01-03  0.456  0.456  0.424  0.456  107078
2 2019-01-04  0.440  0.456  0.416  0.456  211146
3 2019-01-07  0.456  0.464  0.424  0.424   70750
4 2019-01-08  0.448  0.448  0.432  0.440   17355
